In [6]:
from getReplicatesDataFrameDapFromPeaks import(
    generateReplicatesOrder,
    generateMeanReplicatesDf
)

In [ ]:
def performBedGraphextraction(folder):
    with cd(folder):
        file_names = os.listdir()
        for file in file_names:
            if file.endswith('orted.bam'):
                # samsorted = file
                bamsorted = file
                bigw = str(bamsorted[:-10]) + 'coverage.bedgraph'
        # subprocess.run('samtools index ' + bamsorted, shell=True)
        subprocess.run('bamCoverage -b ' + bamsorted +
                        ' -o ' + bigw + 
                        ' -of bigwig --normalizeUsing BPM --binSize 10 --numberOfProcessors 40', shell=True)

In [12]:
specificPathsSumary = generateReplicatesOrder()

replicate3 MYC3/amplified/6/Mock MISSING
replicate2 MYCH7/direct/24/Mock MISSING


In [36]:
import glob
import subprocess
import pandas as pd
allNormalizedreplicates = []
mkdir_toput_reswult = '/home/joaquin/projects/methylation/.vscode/resultado_bedgraph'
from functools import reduce

for experiment in specificPathsSumary:
    if "MYC3" in experiment:
        for metState in specificPathsSumary[experiment]:
            for exptype in specificPathsSumary[experiment][metState]:
                listofdfs = []
                datacolnames = []
                df_merged=None
                for replicate in specificPathsSumary[experiment][metState][exptype]:
                    workingFolder = specificPathsSumary[experiment][metState][exptype][replicate]
                    datacolname = '{}{}{}{}'.format(experiment,metState,exptype,replicate)
                    bedgraph_filename =  glob.glob(workingFolder+'/*.bw')[0][:-11]+'.bedgraph'
                    print(bedgraph_filename,datacolname, glob.glob(workingFolder+'/*orted.bam'))
                    # subprocess.run('bamCoverage -b ' + glob.glob(workingFolder+'/*orted.bam')[0] +
                    #     ' -o ' + bedgraph_filename + 
                    #     ' -of bedgraph --normalizeUsing BPM --binSize 10 --numberOfProcessors 40', shell=True)
                    bedgraph_df = pd.read_csv(bedgraph_filename,sep='\t',names=['chr','star','end',datacolname], dtype={'chr':str})
                    
                    datacolnames.append(datacolname)
                    listofdfs.append(bedgraph_df)
        #             los valores que no estan en una de las replicas los completo con un 0
                df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['chr','star','end'],
                                                how='outer'), listofdfs)
#                 print(df_merged)
                mean_col_name = '{}{}{}'.format(experiment,metState,exptype)
                df_merged[mean_col_name] = df_merged[datacolnames].mean(axis=1)
                df_merged = df_merged.drop(columns=datacolnames)
                bedgraph_df.to_csv(mkdir_toput_reswult+f'/{mean_col_name}dap_seq.bw', sep='\t' ,index=False)


/home/joaquin/projects/methylation/data/tfs_rep_1/MYC3/direct/1/Mock/MYC3direct1Mock.bedgraph MYC31Mockdirectsamplereplicate1 ['/home/joaquin/projects/methylation/data/tfs_rep_1/MYC3/direct/1/Mock/MYC3direct1MockSorted.bam']
/home/joaquin/projects/methylation/data/tfs_rep_3_input_from_rep_2/MYC3/direct/1/Mock/MYC3direct1Mock.bedgraph MYC31Mockdirectsamplereplicate2 ['/home/joaquin/projects/methylation/data/tfs_rep_3_input_from_rep_2/MYC3/direct/1/Mock/MYC3direct1Mocksorted.bam']
/home/joaquin/projects/methylation/data/tfs_rep_4/MYC3/direct/1/Mock/MYC3direct1Mock.bedgraph MYC31Mockdirectsamplereplicate3 ['/home/joaquin/projects/methylation/data/tfs_rep_4/MYC3/direct/1/Mock/MYC3direct1Mocksorted.bam']
/home/joaquin/projects/methylation/data/tfs_rep_1/Input/amplified/1/Mock/Inputamplified1Mock.bedgraph MYC31Mockdirectinputreplicate1 ['/home/joaquin/projects/methylation/data/tfs_rep_1/Input/amplified/1/Mock/Inputamplified1MockSorted.bam']
/home/joaquin/projects/methylation/data/tfs_rep_2/I

In [6]:
import glob 
import subprocess
for list_path in glob.glob('/home/joaquin/projects/methylation/.vscode/resultado_bedgraph/*'):
    # print(list_path)
    # print('tail -n +2 ' + list_path + ' > temp1 && cat temp1 > ' + list_path +
    #       ' && '+ 'bedGraphToBigWig ' + list_path + ' ' + '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/genome.index.txt' + ' ' + list_path[:-8]+'bw')
    subprocess.run('tail -n +2 ' + list_path + ' > temp1 && cat temp1 > ' + list_path +
          ' && '+ 'bedGraphToBigWig ' + list_path + ' ' + '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/genome.index.txt' + ' ' + list_path[:-8]+'bw',
            shell=True)



In [28]:
bedgraph_df = pd.read_csv(bedgraph_filename,sep='\t',names=['chr','star','end',bedgraph_filename])

In [29]:
bedgraph_df

,chr,star,end,/home/joaquin/projects/methylation/data/tfs_rep_1/MYC3/direct/1/Mock/MYC3direct1Mock.bedgraph
0,1,0,40,0.000000
1,1,40,50,0.118197
2,1,50,80,0.236394
3,1,80,120,0.354592
4,1,120,130,0.590986
...,...,...,...,...
4555580,Pt,154430,154440,122.216000
4555581,Pt,154440,154450,104.723000
4555582,Pt,154450,154460,89.475300
4555583,Pt,154460,154470,69.381800
